# Проект - ИИ-ассистент авиакомпании

Теперь мы объединим наши знания, чтобы создать ИИ-ассистента по поддержке клиентов для авиакомпании

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# Эта функция выглядит гораздо проще, чем в моем видео, потому что мы используем последние обновления радио

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Инструменты

Инструменты - это невероятно мощная функция, предоставляемая frontier Lms.

С помощью инструментов вы можете написать функцию и заставить LLM вызывать эту функцию в качестве ответа.

Звучит почти пугающе.. мы даем ей возможность запускать код на нашем компьютере?

Ну, типа того.

In [ ]:
# Давайте начнем с создания полезной функции

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("London")

In [ ]:
# Существует определенная структура словаря, которая требуется для описания нашей функции:

price_function = {
    "name": "get_ticket_price",
    "description": "Узнайте стоимость билета в оба конца до города назначения. Звоните по этому номеру всякий раз, когда вам нужно узнать стоимость билета, например, когда клиент спрашивает: 'Сколько стоит билет до этого города'.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "Город, в который клиент хочет отправиться",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# И это входит в список инструментов:

tools = [{"type": "function", "function": price_function}]

## Как заставить OpenAI использовать наш инструмент

Есть некоторые хитрости, позволяющие OpenAI "вызывать наш инструмент"

На самом деле мы даем LLM возможность сообщить нам, что он хочет, чтобы мы запустили этот инструмент.

Вот как выглядит новая функция чата:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# Мы должны написать эту функцию handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

# Давайте перейдем к мультимодальному режиму!!

Мы можем использовать DALLE3, модель генерации изображений, используемую в GPT-4o, для создания нескольких изображений

Давайте поместим это в функцию под названием artist.

### Предупреждение о цене: каждый раз, когда я создаю изображение, это стоит около 4 центов - не сходите с ума от изображений!

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

## Audio (NOTE - Audio is optional for this course - feel free to skip Audio if it causes trouble!)

And let's make a function talker that uses OpenAI's speech model to generate Audio

### Troubleshooting Audio issues

If you have any problems running this code below (like a FileNotFound error, or a warning of a missing package), you may need to install FFmpeg, a very popular audio utility.

**For PC Users**

Detailed instructions are [here](https://chatgpt.com/share/6724efee-6b0c-8012-ac5e-72e2e3885905) and summary instructions:

1. Download FFmpeg from the official website: https://ffmpeg.org/download.html

2. Extract the downloaded files to a location on your computer (e.g., `C:\ffmpeg`)

3. Add the FFmpeg bin folder to your system PATH:
- Right-click on 'This PC' or 'My Computer' and select 'Properties'
- Click on 'Advanced system settings'
- Click on 'Environment Variables'
- Under 'System variables', find and edit 'Path'
- Add a new entry with the path to your FFmpeg bin folder (e.g., `C:\ffmpeg\bin`)
- Restart your command prompt, and within Jupyter Lab do Kernel -> Restart kernel, to pick up the changes

4. Open a new command prompt and run this to make sure it's installed OK
`ffmpeg -version`

**For Mac Users**

1. Install homebrew if you don't have it already by running this in a Terminal window and following any instructions:  
`/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"`

2. Then install FFmpeg with `brew install ffmpeg`

3. Verify your installation with `ffmpeg -version` and if everything is good, within Jupyter Lab do Kernel -> Restart kernel to pick up the changes

Message me or email me at ed@edwarddonner.com with any problems!

## Аудио (ПРИМЕЧАНИЕ - Аудио для этого курса необязательно - не стесняйтесь пропускать аудио, если оно вызывает проблемы!)

И давайте создадим функцию talker, которая использует речевую модель OpenAI для генерации звука

### Устранение неполадок со звуком

Если у вас возникнут какие-либо проблемы с запуском приведенного ниже кода (например, ошибка FileNotFound или предупреждение об отсутствии пакета), возможно, вам потребуется установить FFmpeg, очень популярную аудиоинтерфейсную утилиту.

**Для пользователей ПК**

Подробные инструкции приведены [здесь](https://chatgpt.com/share/6724efee-6b0c-8012-ac5e-72e2e3885905) и краткие инструкции:

1. Загрузите FFmpeg с официального веб-сайта: https://ffmpeg.org/download.html

2. Извлеките загруженные файлы в папку на вашем компьютере (например, `C:\ffmpeg`)

3. Добавьте папку FFmpeg bin в свой системный путь:
- Щелкните правой кнопкой мыши "Этот компьютер" или "Мой компьютер" и выберите "Свойства"
- Нажмите "Дополнительные системные настройки"
- Нажмите "Переменные среды"
- В разделе "Системные переменные" найдите и отредактируйте "Путь"
- Добавьте новую запись с указанием пути к вашей папке FFmpeg bin (например, `C:\ffmpeg\bin`).
- Перезапустите командную строку и в Jupyter Lab выполните команду Ядро -> Перезапустить ядро, чтобы внести изменения

4. Откройте новую командную строку и запустите ее, чтобы убедиться, что она установлена нормально
`ffmpeg -версия`

**Для пользователей Mac**

1. Установите homebrew, если у вас его еще нет, запустив его в окне терминала и следуя любым инструкциям:  
`/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"`

2. Затем установите FFmpeg с помощью "brew install ffmpeg`

3. Проверьте свою установку с помощью "ffmpeg -версии", и если все в порядке, в лаборатории Jupyter выполните команду "Ядро" -> "Перезапустить ядро", чтобы внести изменения

Напишите мне или напишите по адресу ed@edwarddonner.com при возникновении каких-либо проблем!

## Чтобы проверить, что у вас теперь есть ffmpeg и вы можете получить к нему доступ здесь

Выполните следующую ячейку, чтобы узнать, получите ли вы номер версии. (Поставьте восклицательный знак перед тем, как что-то в Jupyter Lab сообщит, что оно должно выполняться как терминальная команда, а не как код на python).

Если это не сработает, вам, возможно, придется сохранить и закрыть вашу лабораторию Jupyter и запустить ее снова из нового окна терминала (Mac) или из командной строки Anaconda (ПК), не забыв активировать среду llms. Это гарантирует, что вы получите ffmpeg.

И если это не сработает, пожалуйста, свяжитесь со мной!

In [ ]:
!ffmpeg -version
!ffprobe -version
!ffplay -version

# Для пользователей Mac и, возможно, для многих пользователей ПК тоже

Эта версия должна работать нормально. Она может работать и для пользователей Windows, но при записи во временный файл может возникнуть ошибка с правами доступа. Если это так, смотрите следующий раздел!

Как всегда, если у вас возникнут проблемы, пожалуйста, свяжитесь со мной! (Вы также можете прокомментировать функцию audio talker() в более позднем коде, если вас меньше интересует генерация звука)

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
talker("Well, hi there")

# Для пользователей Windows (или любых пользователей Mac с вышеуказанными проблемами)

## Сначала попробуйте версию для Mac, указанную выше, но если вы получите ошибку при записи во временный файл с правами доступа, то вместо этого должен сработать этот код.

Совместная работа студентов Марка М., Патрика Х. и Клода помогла решить эту проблему!

Ниже приведены 4 варианта - надеюсь, один из них подойдет для вашего ПК. Если нет, напишите мне, пожалуйста!

И для пользователей Mac - все 3 из приведенных ниже вариантов также работают на моем Mac - попробуйте их, если у вас возникли проблемы с версией для Mac.

## Вариант 1 для ПК

In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

## PC Variation 2

In [ ]:
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time

def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # Student Dominic found that this was needed. You could also try commenting out to see if not needed on your PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

talker("Well hi there")

## PC Variation 3

In [ ]:
import os
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO

def talker(message):
    # Set a custom directory for temporary files on Windows
    custom_temp_dir = os.path.expanduser("~/Documents/temp_audio")
    os.environ['TEMP'] = custom_temp_dir  # You can also use 'TMP' if necessary
    
    # Create the folder if it doesn't exist
    if not os.path.exists(custom_temp_dir):
        os.makedirs(custom_temp_dir)
    
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    play(audio)

talker("Well hi there")

## PC Variation 4

### Давайте попробуем совершенно другую звуковую библиотеку

Сначала запустите следующую ячейку, чтобы установить новую библиотеку, затем попробуйте ячейку под ней.

In [ ]:
!pip install simpleaudio

In [ ]:
from pydub import AudioSegment
from io import BytesIO
import tempfile
import os
import simpleaudio as sa

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    # Create a temporary file in a folder where you have write permissions
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False, dir=os.path.expanduser("~/Documents")) as temp_audio_file:
        temp_file_name = temp_audio_file.name
        audio.export(temp_file_name, format="wav")
    
    # Load and play audio using simpleaudio
    wave_obj = sa.WaveObject.from_wave_file(temp_file_name)
    play_obj = wave_obj.play()
    play_obj.wait_done()  # Wait for playback to finish

    # Clean up the temporary file afterward
    os.remove(temp_file_name)
    
talker("Well hi there")

## Пользователи ПК - если ни один из этих 4 вариантов не сработал!

Пожалуйста, свяжитесь со мной. Мне жаль, что это вызывает проблемы! Мы разберемся с этим.

В качестве альтернативы: воспроизведение аудио с вашего ПК не является особо важным для этого курса, и вы можете сосредоточиться на создании изображений и пропустить аудио на данный момент или вернуться к нему позже.

# Наша агентная платформа

Термин "Агентный ИИ" и организация - это обобщающий термин, который относится к ряду методов, таких как:

1. Разбиение сложной проблемы на более мелкие этапы с использованием нескольких Lms, выполняющих специализированные задачи
2. Возможность Lms использовать инструменты, которые предоставляют им дополнительные возможности
3. "Агентская среда", позволяющая агентам сотрудничать
4. LLM может выступать в роли планировщика, разделяя большие задачи на более мелкие для специалистов
5. Концепция Агента, обладающего автономией, выходящей за рамки простого ответа на запрос - например, память

Здесь мы показываем 1 и 2, и в меньшей степени 3 и 5. На 8-й неделе мы сделаем многое другое!

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Закомментируйте или удалите следующую строку, если вы предпочитаете пока пропустить аудио..
    talker(reply)
    
    return history, image

In [ ]:
# Более сложный код для радиосвязи, поскольку мы не используем предустановленный интерфейс чата!
# Ввод значения inbrowser=True в последней строке приведет к немедленному появлению окна радиосвязи.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

# Упражнения и бизнес-приложения

Добавьте дополнительные инструменты - возможно, для имитации реального бронирования авиабилета. Один из студентов сделал это и представил свой пример в папке "Вклады сообщества".

Далее: возьмите это и примените к своему бизнесу. Создайте мультимодального помощника с искусственным интеллектом, используя инструменты, которые могли бы выполнять действия, необходимые для вашей работы. Помощник по работе с клиентами? Помощник по адаптации новых сотрудников? Так много возможностей! Кроме того, ознакомьтесь с упражнением в конце второй недели в отдельном блокноте.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">У меня есть к тебе особая просьба</h2>
            <span style="color:#090;">
                Мой редактор говорит мне, что это имеет огромное значение, когда студенты оценивают этот курс на Udemy - это один из основных способов, с помощью которого Udemy решает, показывать ли его другим. Если у вас найдется минутка, чтобы оценить это, я был бы вам очень признателен! И, несмотря ни на что, всегда обращайтесь ко мне по адресу ed@edwarddonner.com если я могу чем-то помочь.
            </span>
        </td>
    </tr>
</table>